<a href="https://colab.research.google.com/github/arampacha/fastai_nbs/blob/main/deberta_finetuning_fastai_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastai transformers ohmeow-blurr wandb

     |████████████████████████████████| 194kB 9.1MB/s 
     |████████████████████████████████| 1.8MB 15.6MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 2.0MB 50.3MB/s 
     |████████████████████████████████| 61kB 9.2MB/s 
     |████████████████████████████████| 3.2MB 49.9MB/s 
     |████████████████████████████████| 890kB 52.1MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 4.1MB 48.9MB/s 
     |████████████████████████████████| 184kB 58.0MB/s 
     |████████████████████████████████| 14.5MB 226kB/s 
     |████████████████████████████████| 1.2MB 51.1MB/s 
     |████████████████████████████████| 163kB 57.6MB/s 
     |████████████████████████████████| 102kB 15.4MB/s 
     |████████████████████████████████| 133kB 56.4MB/s 
     |████████████████████████████████| 102kB 14.1MB/s 
     |████████████████████████████████| 20.7MB 1.4MB/s 
     |████████████████████████████████| 245kB 57.1MB/s 

In [ ]:
from fastai.text.all import *
from fastai.callback.wandb import *

In [ ]:
def read_text(fn):
    return open(fn).read()

In [ ]:
path = untar_data(URLs.IMDB)

In [ ]:
model_name = "microsoft/deberta-base"

max_len = 512
bs = 8
val_bs = 16

## Tracking

In [ ]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb

WANDB_NAME = f'imdb-{model_name}-simple'
GROUP = 'IMDB'
NOTES = f'Simple finetuning {model_name} with RAdam lr=2e-5'
CONFIG = {}
TAGS =[model_name,'imdb','radam','simple']

In [ ]:
wandb.init(reinit=True, project="lm-finetuning", entity="arampacha", 
           name=WANDB_NAME, group=GROUP, notes=NOTES, tags=TAGS, config=CONFIG)

wandb: Currently logged in as: fastai_community (use `wandb login --relogin` to force relogin)


## Training

In [ ]:
from transformers import *

from blurr.data.all import *
from blurr.modeling.all import *

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
task = HF_TASKS_AUTO.SequenceClassification

hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(model_name, task=task, tokenizer_kwargs={'max_len':512})

In [ ]:
blocks = (HF_TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), CategoryBlock)
dblock = DataBlock(blocks=blocks, 
                   get_items=get_text_files,
                   get_x = read_text,
                   get_y=parent_label,
                   splitter=GrandparentSplitter(valid_name='test'))

dls = dblock.dataloaders(path, bs=bs, val_bs=val_bs)

In [ ]:
model = HF_BaseModelWrapper(hf_model)
learn = Learner(dls,
                model,
                opt_func=RAdam,
                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter).to_fp16()

learn.create_opt() 
learn.freeze()
learn.blurr_summary()

HF_BaseModelWrapper (Input shape: 8 x 512)
Layer (type)         Output Shape         Param #    Trainable 
                     8 x 512 x 768       
Embedding                                 38603520   False     
DebertaLayerNorm                          1536       False     
StableDropout                                                  
____________________________________________________________________________
                     8 x 512 x 2304      
Linear                                    1769472    False     
StableDropout                                                  
Linear                                    589824     False     
Linear                                    590592     False     
StableDropout                                                  
Linear                                    590592     False     
DebertaLayerNorm                          1536       False     
StableDropout                                                  
____________________________

### Simple ft

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(4, 2e-5, cbs=WandbCallback(log_preds=False, log_model=False))

Could not gather input dimensions


epoch,train_loss,valid_loss,accuracy,time
0,0.160276,0.164386,0.939760,29:55
1,0.132156,0.129645,0.953120,30:00
2,0.048035,0.143527,0.957000,30:00
3,0.044810,0.180624,0.958000,29:58


### All unfrozen

In [ ]:
learn.load('stage0')
learn.unfreeze()
learn.fit_one_cycle(3, slice(2e-6, 2e-5))

epoch,train_loss,valid_loss,accuracy,time
0,0.184030,0.191095,0.925880,16:47
1,0.179359,0.179827,0.931040,16:47
2,0.120620,0.194259,0.931000,16:46


### Adam

In [ ]:
learn.load('stage0')
learn.opt_func = Adam

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, 2e-5)

epoch,train_loss,valid_loss,accuracy,time
